<a href="https://colab.research.google.com/github/aannddrree/disciplinaIA/blob/main/ALgoritmoEndereco.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# arquivo: treino_logistico_enderecos.py
# Requisitos (pip):
#   pandas scikit-learn joblib numpy

import pandas as pd
import numpy as np
from pathlib import Path

from sklearn.model_selection import StratifiedGroupKFold
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, average_precision_score, f1_score
from sklearn.exceptions import UndefinedMetricWarning
import warnings
import joblib

warnings.filterwarnings("ignore", category=UndefinedMetricWarning)

# =========================
# 1) Carregar dados
# =========================
CSV_PATH = "/content/enderecos_com_score.csv"  # ajuste se necessário
df = pd.read_csv(CSV_PATH)

# Checagens básicas
required_cols = {"endereco_completo", "cidade", "estado", "lat", "lon", "score"}
missing = required_cols - set(df.columns)
if missing:
    raise ValueError(f"Colunas faltantes no CSV: {missing}")

# =========================
# 2) Criar alvo binário com salvaguardas
#    - Garante pelo menos 2 classes
#    - Tenta quantis; cai para mediana se necessário
#    - Também verifica mínimo de positivos
# =========================
def make_binary_labels_from_score(scores: pd.Series, prefer_q=0.7, min_pos=10):
    qs = [prefer_q, 0.75, 0.8, 0.65, 0.6, 0.55, 0.5, 0.45, 0.4]  # tenta vários quantis
    for q in qs:
        thr = scores.quantile(q)
        y = (scores >= thr).astype(int)
        if y.nunique() == 2 and y.sum() >= min_pos:
            return y, float(thr), q
    # fallback final: mediana
    thr = scores.median()
    y = (scores >= thr).astype(int)
    return y, float(thr), None

y, thr, used_q = make_binary_labels_from_score(df["score"])
print(f"[INFO] Limiar usado p/ classe positiva: {thr:.2f} "
      f"(quantil: {used_q if used_q is not None else 'mediana'})")
print("[INFO] Distribuição de classes:", y.value_counts().to_dict())

# =========================
# 3) Features e pré-processamento
# =========================
text_col = "endereco_completo"
cat_cols  = ["cidade", "estado"]
num_cols  = ["lat", "lon"]  # adicione outras numéricas se tiver

preprocess = ColumnTransformer(
    transformers=[
        ("txt", TfidfVectorizer(max_features=5000, ngram_range=(1, 2)), text_col),
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
        ("num", StandardScaler(), num_cols),
    ],
    remainder="drop",
    verbose_feature_names_out=False
)

clf = LogisticRegression(
    solver="lbfgs",
    max_iter=2000,
    class_weight="balanced",
    n_jobs=None  # remova se sua versão do sklearn não suportar
)

pipe = Pipeline([("prep", preprocess), ("clf", clf)])

X = df[[text_col] + cat_cols + num_cols]
groups = df["cidade"].astype(str)

# =========================
# 4) Validação (StratifiedGroupKFold)
#    - Estratifica por y, mas respeita grupos (cidade)
# =========================
n_splits = 5
sgkf = StratifiedGroupKFold(n_splits=n_splits, shuffle=True, random_state=42)

aucs, aps, f1s = [], [], []
split_id = 1
for tr_idx, te_idx in sgkf.split(X, y, groups):
    X_tr, X_te = X.iloc[tr_idx], X.iloc[te_idx]
    y_tr, y_te = y.iloc[tr_idx], y.iloc[te_idx]

    # Segurança extra: se por acaso a dobra ainda ficar mono-classe
    if y_tr.nunique() < 2 or y_te.nunique() < 2:
        print(f"[WARN] Dobra {split_id} sem 2 classes em treino ou teste. Pulando.")
        split_id += 1
        continue

    pipe.fit(X_tr, y_tr)
    p = pipe.predict_proba(X_te)[:, 1]
    yhat = (p >= 0.5).astype(int)

    aucs.append(roc_auc_score(y_te, p))
    aps.append(average_precision_score(y_te, p))
    f1s.append(f1_score(y_te, yhat))
    print(f"[Fold {split_id}] AUC={aucs[-1]:.4f} | AP={aps[-1]:.4f} | F1={f1s[-1]:.4f}")
    split_id += 1

if len(aucs) == 0:
    raise RuntimeError("Todas as dobras foram inválidas (mono-classe). "
                       "Revise sua base ou a regra de rotulagem.")

print(f"[CV] AUC  médio: {np.mean(aucs):.4f} ± {np.std(aucs):.4f}")
print(f"[CV] AP   médio: {np.mean(aps):.4f} ± {np.std(aps):.4f}")
print(f"[CV] F1   médio: {np.mean(f1s):.4f} ± {np.std(f1s):.4f}")

# =========================
# 5) Treino final no dataset completo
# =========================
pipe.fit(X, y)

# =========================
# 6) Salvar artefatos do modelo e metadados
# =========================
OUT_DIR = Path("artefatos_modelo")
OUT_DIR.mkdir(exist_ok=True)

joblib.dump(pipe, OUT_DIR / "modelo_logistico_endereco.joblib")
meta = {
    "threshold": thr,
    "used_quantile": used_q,
    "text_col": text_col,
    "cat_cols": cat_cols,
    "num_cols": num_cols,
    "n_splits": n_splits,
    "cv_auc_mean": float(np.mean(aucs)),
    "cv_ap_mean": float(np.mean(aps)),
    "cv_f1_mean": float(np.mean(f1s)),
}
pd.Series(meta, dtype=object).to_json(OUT_DIR / "metadata.json", force_ascii=False)
print(f"[OK] Modelo salvo em: {OUT_DIR / 'modelo_logistico_endereco.joblib'}")
print(f"[OK] Metadata salva em: {OUT_DIR / 'metadata.json'}")

# =========================
# 7) Exemplo de ranqueamento de candidatos
# =========================
candidatos = pd.DataFrame([
    {"endereco_completo": "Av. Paulista, 1000, Bela Vista",
     "cidade": "Sao Paulo", "estado": "SP", "lat": -23.5614, "lon": -46.6559},
    {"endereco_completo": "Rua Exemplo, 123, Centro",
     "cidade": "Sao Paulo", "estado": "SP", "lat": -23.5440, "lon": -46.6330},
    {"endereco_completo": "Av. Atlântica, 3000, Copacabana",
     "cidade": "Rio de Janeiro", "estado": "RJ", "lat": -22.9717, "lon": -43.1830},
])

# Atenção: as colunas dos candidatos devem bater com X (mesmas features)
candidatos["prob_bom"] = pipe.predict_proba(candidatos)[:, 1]
print("\n[Ranking de candidatos por probabilidade de 'bom']\n",
      candidatos.sort_values("prob_bom", ascending=False).to_string(index=False))


[INFO] Limiar usado p/ classe positiva: 64.31 (quantil: mediana)
[INFO] Distribuição de classes: {1: 5, 0: 5}
[Fold 1] AUC=1.0000 | AP=1.0000 | F1=0.4000
[WARN] Dobra 2 sem 2 classes em treino ou teste. Pulando.
[Fold 3] AUC=0.0000 | AP=0.5000 | F1=0.0000
[Fold 4] AUC=0.0000 | AP=0.5000 | F1=0.6667
[WARN] Dobra 5 sem 2 classes em treino ou teste. Pulando.
[CV] AUC  médio: 0.3333 ± 0.4714
[CV] AP   médio: 0.6667 ± 0.2357
[CV] F1   médio: 0.3556 ± 0.2740
[OK] Modelo salvo em: artefatos_modelo/modelo_logistico_endereco.joblib
[OK] Metadata salva em: artefatos_modelo/metadata.json

[Ranking de candidatos por probabilidade de 'bom']
               endereco_completo         cidade estado      lat      lon  prob_bom
Av. Atlântica, 3000, Copacabana Rio de Janeiro     RJ -22.9717 -43.1830  0.631894
 Av. Paulista, 1000, Bela Vista      Sao Paulo     SP -23.5614 -46.6559  0.505657
       Rua Exemplo, 123, Centro      Sao Paulo     SP -23.5440 -46.6330  0.481003
